In [61]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [54]:
#Parsing Training documents
L_train = pd.read_parquet('book_train.parquet/stock_id=0')
df_train = pd.read_csv('train.csv')


In [55]:
#now to create a function that works out the Weighted average price 
def Wap(df):
    # Formula: (BidPrice * AskSize + AskPrice * BidSize) / TotalSize
    a=df['bid_price1']*df['ask_size1']
    b=df['ask_price1']*df['bid_size1']
    c=df['ask_size1']+df['bid_size1']
    return (a+b)/c
L_train['wap'] = Wap(L_train)


In [56]:
def Log_rets(x):
    #This works out ln(current/previous price)
    return np.log(x).diff()

In [57]:
#Getting Log Returns 
L_train['log_returns'] = L_train.groupby('time_id')['wap'].transform(Log_rets)
L_train=L_train.dropna()


In [58]:
#Now to caluclate realised volatility
#I found HFT the mean should be considered 0 so i updated the formula to reflect th\t
def Realisedvol(x):
    return np.sqrt(np.sum(x**2))

df_features = L_train.groupby('time_id').agg(volatility_feature=('log_returns',Realisedvol)).reset_index()
print(df_features.head())

   time_id  volatility_feature
0        5            0.004499
1       11            0.001204
2       16            0.002369
3       31            0.002574
4       62            0.001894


In [ ]:
#creating a final dataframe for stock 0
stock0_target = df_train[df_train['stock_id']==0]
df_final = pd.merge(stock0_target, df_features,on='time_id',how='inner')
print(df_final.head())

   stock_id  time_id    target  volatility_feature
0         0        5  0.004136            0.004499
1         0       11  0.001445            0.001204
2         0       16  0.002168            0.002369
3         0       31  0.002195            0.002574
4         0       62  0.001747            0.001894
